In [1]:
import requests
import zipfile
import numpy as np
import tensorflow as tf

tf.config.run_functions_eagerly(True)

In [2]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
# Getting the data 1
url = 'https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip'
dataset_path = '../homework8-deep-learning/datasets/'
response = requests.get(url)
with open(f'{dataset_path}data.zip', 'wb') as file:
    for chunk in response.iter_content(chunk_size=1024):
        file.write(chunk)

In [4]:
with zipfile.ZipFile(f'{dataset_path}data.zip') as zip_ref:
    zip_ref.extractall(f'{dataset_path}/hairs-dataset')

In [5]:
# Define CNN the model

# noinspection PyUnresolvedReferences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(Input(shape=(200, 200, 3)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
optimizer = SGD(learning_rate=0.002, momentum=0.8)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 313632)            0         
                                                                 
 dense (Dense)               (None, 64)                20072512  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 20073473 (76.57 MB)
Trainable params: 20073473 (76.57 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [6]:
# Create the data generator, load data

# noinspection PyUnresolvedReferences
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rescale=1. / 255)

train_dataset = gen.flow_from_directory(
    '../homework8-deep-learning/datasets/hairs-dataset/data/train',
    target_size=(200, 200),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

test_dataset = gen.flow_from_directory(
    '../homework8-deep-learning/datasets/hairs-dataset/data/test',
    target_size=(200, 200),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [7]:
# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10


/Users/roman_didyk/.local/share/virtualenvs/rd-ml-zoomcamp-2024-3prO3anl/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


40/40 [==============================] - 8s 205ms/step - loss: 0.6915 - accuracy: 0.5863 - val_loss: 0.6552 - val_accuracy: 0.5721
Epoch 2/10
40/40 [==============================] - 6s 147ms/step - loss: 0.6008 - accuracy: 0.6775 - val_loss: 0.6299 - val_accuracy: 0.6368
Epoch 3/10
40/40 [==============================] - 6s 151ms/step - loss: 0.5884 - accuracy: 0.6913 - val_loss: 0.6132 - val_accuracy: 0.6468
Epoch 4/10
40/40 [==============================] - 6s 147ms/step - loss: 0.5473 - accuracy: 0.7200 - val_loss: 0.6491 - val_accuracy: 0.6368
Epoch 5/10
40/40 [==============================] - 6s 147ms/step - loss: 0.5340 - accuracy: 0.7225 - val_loss: 0.6379 - val_accuracy: 0.6418
Epoch 6/10
40/40 [==============================] - 6s 149ms/step - loss: 0.5068 - accuracy: 0.7475 - val_loss: 0.6061 - val_accuracy: 0.6766
Epoch 7/10
40/40 [==============================] - 6s 148ms/step - loss: 0.4892 - accuracy: 0.7513 - val_loss: 0.5992 - val_accuracy: 0.6617
Epoch 8/10
40/40 

In [8]:
train_accuracies = history.history['accuracy']
median_accuracy = np.median(train_accuracies)
median_accuracy

0.7268750071525574

In [9]:
train_losses = history.history['loss']
std_dev_loss = np.std(train_losses)
std_dev_loss

0.07614932526635858

In [10]:
gen_with_augmentation = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_dataset_with_augmentation = gen_with_augmentation.flow_from_directory(
    '../homework8-deep-learning/datasets/hairs-dataset/data/train',
    target_size=(200, 200),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

test_dataset_with_augmentation = gen_with_augmentation.flow_from_directory(
    '../homework8-deep-learning/datasets/hairs-dataset/data/test',
    target_size=(200, 200),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [11]:
history_with_augmentation = model.fit(train_dataset_with_augmentation, epochs=10, validation_data=test_dataset_with_augmentation)

Epoch 1/10
40/40 [==============================] - 7s 182ms/step - loss: 0.6582 - accuracy: 0.6438 - val_loss: 0.6481 - val_accuracy: 0.6716
Epoch 2/10
40/40 [==============================] - 7s 170ms/step - loss: 0.6119 - accuracy: 0.6650 - val_loss: 0.6131 - val_accuracy: 0.6766
Epoch 3/10
40/40 [==============================] - 7s 172ms/step - loss: 0.5906 - accuracy: 0.6925 - val_loss: 0.6026 - val_accuracy: 0.6716
Epoch 4/10
40/40 [==============================] - 7s 171ms/step - loss: 0.5957 - accuracy: 0.6913 - val_loss: 0.6131 - val_accuracy: 0.6617
Epoch 5/10
40/40 [==============================] - 7s 172ms/step - loss: 0.5832 - accuracy: 0.6963 - val_loss: 0.5960 - val_accuracy: 0.6915
Epoch 6/10
40/40 [==============================] - 7s 172ms/step - loss: 0.5862 - accuracy: 0.7050 - val_loss: 0.6264 - val_accuracy: 0.6418
Epoch 7/10
40/40 [==============================] - 7s 175ms/step - loss: 0.5742 - accuracy: 0.7075 - val_loss: 0.5948 - val_accuracy: 0.6617
Epoch 

In [12]:
test_losses = history_with_augmentation.history['val_loss']
mean_test_loss = np.mean(test_losses)
mean_test_loss

0.6003234505653381

In [13]:
test_accuracies = history_with_augmentation.history['val_accuracy'][-5:]
average_test_accuracy = np.mean(test_accuracies)
average_test_accuracy

0.68855721950531